In [1]:
import numpy as np
import pandas as pd
import locale
import time
#import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


locale.setlocale(locale.LC_TIME, "de_DE")

%store -r columnNames

#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199

# pd.describe_option('display')
#plt.close("all")

In [2]:
# Create color set bades on UR CD colors
urCdColors = [
    "rgb(156,0,75)", # HEIDENELKENROT
    "rgb(0,155,119)", # TÜRKISGRÜN
    "rgb(0,135,178)", # SPEKTRALBLAU
    "rgb(174,167,0)", # LÄRCHENNADELGRÜN
    "rgb(236,98,0)", # ORANGEROT
    "rgb(191,0,42)", # HEUCHERAROT
    "rgb(79,184,0)", # BLATTGRÜN
    "rgb(0,137,147)", # EISVOGELBLAU
    "rgb(0,85,106)", # CAPRIBLAU
    "rgb(205,211,15)", # URANGELB
    "rgb(236,188,0)" # VATIKANGELB
]

# Go table header colors
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

# Get Results

In [3]:
surveyData=pd.read_csv('justCompletedAttempts.csv', sep=';')
print("Anzahl an kompletten Daten:", len(surveyData))

# Convert to seconds
def secondsToTime(seconds): 
    timeString =  str( time.strftime('%H:%M:%S', time.gmtime(seconds)) ) + str( seconds )[-3:]
    return timeString

# Converts mood value in float
def moodToFloatVal(moodString):
    moodFloat = 0.0

    if moodString == "Schlecht":
        moodFloat = -2.0
    elif moodString == "Nicht gut":
        moodFloat = -1.0
    elif moodString == "Neutral":
        moodFloat = 0.0
    elif moodString == "Gut":
        moodFloat = 1.0
    elif moodString == "Sehr gut":
        moodFloat = 2.0
        
    return moodFloat

# Add extra fields for comparison
surveyData['duration-over-all-time'] = surveyData.apply(lambda row: secondsToTime(row['duration-over-all']), axis=1)
surveyData['duration-date-based'] = surveyData.apply(lambda row: pd.to_datetime(row['date-last'])-pd.to_datetime(row['date-start']), axis=1)
surveyData['duration-date-based-in-seconds'] = surveyData.apply(lambda row: row['duration-date-based'].total_seconds(), axis=1)
surveyData['duration-delta-all-vs-start-end'] = surveyData.apply(lambda row: row['duration-over-all']-row['duration-date-based-in-seconds'], axis=1)
surveyData['duration-shortest-val'] = surveyData.apply(lambda row: min(row['duration-over-all'], row['duration-date-based-in-seconds']), axis=1)
surveyData['duration-shortest-val-time'] = surveyData.apply(lambda row: secondsToTime(row['duration-shortest-val']), axis=1)
surveyData['mood-start-val'] = surveyData.apply(lambda row: moodToFloatVal(row['mood-start']), axis=1)
surveyData['mood-end-val'] = surveyData.apply(lambda row: moodToFloatVal(row['mood-end']), axis=1)
surveyData['mood-delta'] = surveyData.apply(lambda row: row['mood-end-val']-row['mood-start-val'], axis=1)

# Build groups depending on what viz participant had to solve first

listFirstParticipants=surveyData[surveyData['list-question-number']==6]
chordFirstParticipants=surveyData[surveyData['chord-question-number']==6]
mapFirstParticipants=surveyData[surveyData['map-question-number']==6]

Anzahl an kompletten Daten: 203


## Socio-Demographic Data
### Age

In [4]:
df = surveyData[['age', 'highest-preferenced-viz']]

# Label strings
chartLabel = f"Altersverteilung, n={len(df)}"
categoryGroups = ["Alter gesamt", "nach Listen Präferenz", "nach Chord Präferenz", "nach Map Präferenz"]
yLabel = "Alter in Jahren"
fig = go.Figure()

# Age over all
fig.add_trace(go.Box(y=df['age'], name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Age preferred List
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Liste"]['age'], name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Age preferred Chord
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Chord"]['age'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Age preferred Map
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Map"]['age'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.update_layout(
    title=chartLabel,
    xaxis_title="Alter nach Präferenz",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.show()

# Table age
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [42, 32, 22, 20, 18],
      [33, 25.5, 22, 20, 18],
      [57, 36, 24, 21, 18],
      [43, 30, 22, 20, 18]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Gender

In [5]:
df = surveyData[['sex']]

df.fillna('nicht angegeben', inplace=True)
df = df.groupby(['sex'], dropna=False)['sex'].count().reset_index(name='count')

# Label strings
chartLabel = f"Verteiltung nach Geschlecht, n={len(surveyData[['sex']])}"
categoryGroups = ["Männlich", "Keine Angabe", "Weiblich"]

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'])])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table sex
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Geschlecht</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", "<b>Gesamt</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [len(surveyData[surveyData['sex'] == 'männlich'])],
        [surveyData['sex'].isnull().sum()],
        [len(surveyData[surveyData['sex'] == 'weiblich'])],
        [len(surveyData[['sex']])]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

/var/folders/4w/b6kqqdv12hzcznys6kqm6yyr0000gn/T/ipykernel_15193/3261097828.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Education

In [6]:
df = surveyData[['education']]
df = df.groupby(['education'])['education'].count().reset_index(name='count')

# Label strings
chartLabel = f"Verteilung nach Ausbildung, n={len(surveyData[['education']])}"
categoryGroups = ["Qualifizierender Hauptschulabschluss", "Mittlere Reife", "Hochschulreife", "Studium", "Promotion"]
xLabel = "Höchster Bildungsabschluss"
yLabel = "Anzahl"

data = [
    df.loc[df['education'] == categoryGroups[0], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[1], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[2], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[3], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[4], 'count'].iloc[0],
] 

# print(df[['education'] == categoryGroups[0]].count)
print(df.loc[df['education'] == categoryGroups[0], 'count'].iloc[0])

fig = px.bar(df, x=categoryGroups, y=data, text_auto='.0s',
            labels={'x':'Anzahl', 'y':'Grad des Bildungsabschluss'},
            title=chartLabel,
            )
fig.update_traces(marker_color=urCdColors, opacity=0.75)
fig.show()

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Bildungsgrad nach Abschluss</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [data[0]],
        [data[1]],
        [data[2]],
        [data[3]],
        [data[4]]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

4


### Profession

In [7]:
df = surveyData[['profession']]

# print(df)

# Replacements
pd.options.mode.chained_assignment = None  # default='warn'
df.loc[df['profession'].str.contains('student', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Studium', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Studierend', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Medientechnik und Produktion', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Barkeeper', case=False), 'profession'] = 'Barkeeper:in'
df.loc[df['profession'].str.contains('ingenieur', case=False), 'profession'] = 'Ingenieur:in'
df.loc[df['profession'].str.contains('Dipl.Ing.grad', case=False), 'profession'] = 'Ingenieur:in'
df.loc[df['profession'].str.contains('Software', case=False), 'profession'] = 'Software Entwickler:in'
df.loc[df['profession'].str.contains('Mediengestalt', case=False), 'profession'] = 'Mediengestalter:in'
df.loc[df['profession'].str.contains('Ehrenamt', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Arbeitslos', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('-', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('/', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Keinen', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Nicht vorhanden', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Beamt', case=False), 'profession'] = 'Beamtin/Beamter'
df.loc[df['profession'].str.contains('rzt', case=False), 'profession'] = 'Ärtzin/Arzt'
df.loc[df['profession'].str.contains('Kieferorthopädin', case=False), 'profession'] = 'Ärtzin/Arzt'
df.loc[df['profession'].str.contains('Künstler', case=False), 'profession'] = 'Künstler:in'
df.loc[df['profession'].str.contains('Bäderbetriebe', case=False), 'profession'] = 'Bademeister:in'
df.loc[df['profession'].str.contains('Bademeister:in', case=False), 'profession'] = 'Fachangestellte:r für Bäderbetriebe'
df.loc[df['profession'].str.contains('^Angestellte', case=False), 'profession'] = 'Angestellte/Angestellter'
df.loc[df['profession'].str.contains('Systemadministrator', case=False), 'profession'] = 'IT Systemadministrator:in'
df.loc[df['profession'].str.contains('senschaftlicher Mitarb', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('senschaftliche Mitarb', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('wiissenschaftliche', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('wiss. Mitarb', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('Bankfachwirtin', case=False), 'profession'] = 'Bankfachwirt:in'
df.loc[df['profession'].str.contains('Beratung', case=False), 'profession'] = 'Berater:in'
df.loc[df['profession'].str.contains('Bibliothekar', case=False), 'profession'] = 'Bibliothekar:in'
df.loc[df['profession'].str.contains('Briefzusteller', case=False), 'profession'] = 'Briefzusteller:in/Postbot:in'
df.loc[df['profession'].str.contains('postbot', case=False), 'profession'] = 'Briefzusteller:in/Postbot:in'
df.loc[df['profession'].str.contains('Buchhändler', case=False), 'profession'] = 'Buchhändler:in'
df.loc[df['profession'].str.contains('Doktorand', case=False), 'profession'] = 'Doktorand:in'
df.loc[df['profession'].str.contains('kauffrau', case=False), 'profession'] = 'Kauffrau/Kaufmann'
df.loc[df['profession'].str.contains('Fachverkäuferin im Lebensmittelhandwerk', case=False), 'profession'] = 'Fachverkäufer:in im Lebensmittelhandwerk'
df.loc[df['profession'].str.contains('Fremdsprachenkorrespondentin', case=False), 'profession'] = 'Fremdsprachenkorrespondent:in'
df.loc[df['profession'].str.contains('Informatiker', case=False), 'profession'] = 'Informatiker:in'
df.loc[df['profession'].str.contains('Kassiererin', case=False), 'profession'] = 'Kassierer:in'
df.loc[df['profession'].str.contains('Landwirt', case=False), 'profession'] = 'Landwirt:in'
df.loc[df['profession'].str.contains('Manager', case=False), 'profession'] = 'Manager:in'
df.loc[df['profession'].str.contains('Lehrer', case=False), 'profession'] = 'Lehrer:in'
df.loc[df['profession'].str.contains('Mechaniker', case=False), 'profession'] = 'Mechaniker:in/Mechatroniker:in'
df.loc[df['profession'].str.contains('Mechatroniker', case=False), 'profession'] = 'Mechaniker:in/Mechatroniker:in'
df.loc[df['profession'].str.contains('Lektorin', case=False), 'profession'] = 'Lektorin:in'
df.loc[df['profession'].str.contains('Musikvermittlerin', case=False), 'profession'] = 'Musikvermittler:in'
df.loc[df['profession'].str.contains('Professor', case=False), 'profession'] = 'Professor:in'
df.loc[df['profession'].str.contains('Projektassistenz', case=False), 'profession'] = 'Projektassistent:in'
df.loc[df['profession'].str.contains('Referent', case=False), 'profession'] = 'Referent:in'
df.loc[df['profession'].str.contains('Sachbearbeiterin', case=False), 'profession'] = 'Sachbearbeiter:in'
df.loc[df['profession'].str.contains('Schriftsteller', case=False), 'profession'] = 'Schriftsteller:in und Literaturwissenschaftler:in'
df.loc[df['profession'].str.contains('Sekretärin', case=False), 'profession'] = 'Sekretär:in'
df.loc[df['profession'].str.contains('Verwaltungsmitarbeiter', case=False), 'profession'] = 'Verwaltungsmitarbeiter:in'
df.loc[df['profession'].str.contains('Vertriebsaußendienst', case=False), 'profession'] = 'Vertriebsaußendienstler:in'
df.loc[df['profession'].str.contains('Selbstständiger Fotograf', case=False), 'profession'] = 'Fotograf:in'
pd.options.mode.chained_assignment = 'warn'  # default='warn'
#df
unifiedProfessionsCounts = df.groupby(['profession'])['profession'].count().reset_index(name='count')
sortedProfessions = unifiedProfessionsCounts.sort_values(['count', 'profession'], ascending=[False, True])
# sortedProfessions = unifiedProfessionsCounts.sort_values(['count'], ascending=False)

# print(len(sortedProfessions))

# Label strings
chartLabel = f"Verteilung nach Berufen, n={len(surveyData[['education']])}"
categoryGroups = ["Beruf", "Anzahl"]
xLabel = "Eigenschaft"
yLabel = "Beruf"

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=[f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[sortedProfessions.profession, sortedProfessions['count']],
  #     # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
  #       ['<b>Anzahl</b>'],
  #       [data[0]],
  #       [data[1]],
  #       [data[2]],
  #       [data[3]],
  #       [data[4]]
  #     ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*21],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    # legend_title="Legend Title",
    height=1100,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()



### Feeling Start And End

In [8]:
df = surveyData[['mood-start', 'mood-start-val', 'mood-end', 'mood-end-val', 'mood-delta', 'highest-preferenced-viz', 'duration-date-based', 'duration-over-all-time', 'duration-shortest-val']]

moodNeutral = df.loc[df['mood-delta'] == 0]
moodImproved = df.loc[df['mood-delta'] >= 1]
moodBoosted = df.loc[df['mood-delta'] > 1]
moodDecreased = df.loc[df['mood-delta'] <= -1]
moodKilled = df.loc[df['mood-delta'] < -1]

print("Neutrale Entwicklung", len(moodNeutral))
print("Verbesserung", len(moodImproved))
print("Mood-Booster", len(moodBoosted))
print("Verschlechterung", len(moodDecreased))
print("Mood-Killer", len(moodKilled))

Neutrale Entwicklung 145
Verbesserung 35
Mood-Booster 2
Verschlechterung 23
Mood-Killer 3


### Standard Deviation of Test Time

In [9]:
# Standard deviation time and mood
# df = surveyData[['duration-shortest-val', 'mood-delta']]
df = surveyData.loc[surveyData['duration-shortest-val'] < 6000][['duration-shortest-val', 'mood-delta']]

# Label strings
chartLabel = f"Zeitverteilung, n={len(df)} von 203, t < 100 Minuten"
categoryGroups = ["alle Teilnehmer:innen", "stabile Gefühlsentwicklung", "Verbesserung d. Gefühlsentwicklung", "Verschlechterung d. Gefühlsentwicklung"]
yLabel = "Dauer des Tests in Sekunden"
fig = go.Figure()

# Age over all
fig.add_trace(go.Box(y=df['duration-shortest-val'], name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Age preferred List
fig.add_trace(go.Box(y=df[df['mood-delta'] == 0]['duration-shortest-val'], name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Age preferred Chord
fig.add_trace(go.Box(y=df[df['mood-delta'] >= 1]['duration-shortest-val'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Age preferred Map
fig.add_trace(go.Box(y=df[df['mood-delta'] <= -1]['duration-shortest-val'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.update_layout(
    title=chartLabel,
    xaxis_title="Zeitverteilung nach Gefühlsentwicklung",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.show()

# Table age
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      # TODO: GET DIRECT VALUES
      [4367, 2505, 2015, 1253, 319],
      [4083,2494, 1983, 1272, 424],
      [3682, 2919, 2213, 1669, 319],
      [4414, 2350, 1312, 818, 393]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## Mood Development

In [30]:
df = surveyData[['duration-shortest-val', 'mood-delta']]

df = df.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

# Label strings
chartLabel = f"Gefühlsentwicklung der Teilnehmer:innen, n={len(surveyData[['mood-delta']])}"
categoryGroups = ["Verschlechterung um 3 Grade", "Verschlechterung um 2 Grade", "Verschlechterung um 1 Grad", "Stabile Entwicklung", "Verbesserung um 1 Grad", "Verbesserung um 2 Grade"]

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False)])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Gefühlsentwicklung</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>", f"<b>{categoryGroups[5]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [df.iloc[0]['count']],
        [df.iloc[1]['count']],
        [df.iloc[2]['count']],
        [df.iloc[3]['count']],
        [df.iloc[4]['count']],
        [df.iloc[5]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Mood Development To Choice

In [40]:
dfAll = surveyData[['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfAll = dfAll.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

dfListe = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Liste'][['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfListe = dfListe.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

dfChord = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Chord'][['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfChord = dfChord.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

dfMap = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Map'][['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfMap = dfMap.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

# Label strings
chartLabel = f"Gefühlsentwicklung der Teilnehmer:innen, n={len(surveyData[['mood-delta']])}"
categoryGroups = ["Verschlechterung um 3 Grade", "Verschlechterung um 2 Grade", "Verschlechterung um 1 Grad", "Stabile Entwicklung", "Verbesserung um 1 Grad", "Verbesserung um 2 Grade"]

fig = make_subplots(rows=2, cols=3, 
    specs=[
        [None, {'type':'domain'}, None],
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
    ],
    subplot_titles=("Gesamt","Liste favorisiert", "Chord  favorisiert",  "Map favorisiert"))

fig.add_trace(go.Pie(labels=categoryGroups, values=dfAll['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 2)
fig.add_trace(go.Pie(labels=categoryGroups, values=dfListe['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 1)
fig.add_trace(go.Pie(labels=[categoryGroups[0], categoryGroups[2],categoryGroups[3], categoryGroups[4], categoryGroups[5]], values=dfChord['count'], sort=False, direction='clockwise', marker_colors=[urCdColors[0], urCdColors[2], urCdColors[3], urCdColors[4], urCdColors[5]]),
              2, 2)
fig.add_trace(go.Pie(labels=[categoryGroups[2], categoryGroups[3], categoryGroups[4], categoryGroups[5]], values=dfMap['count'], sort=False, direction='clockwise', marker_colors=[urCdColors[2], urCdColors[3], urCdColors[4], urCdColors[5]]),
              2, 3)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    height=850,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
fig.update_annotations(yshift=70)
fig.show()


# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Gefühlsentwicklung</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>", f"<b>{categoryGroups[5]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Gesamt</b>','<b>Liste</b>', '<b>Chord</b>', '<b>Map</b>'],
        [dfAll.iloc[0]['count'], dfListe.iloc[0]['count'], dfChord.iloc[0]['count'], 0],
        [dfAll.iloc[1]['count'], dfListe.iloc[1]['count'], 0, 0],
        [dfAll.iloc[2]['count'], dfListe.iloc[2]['count'], dfChord.iloc[1]['count'], dfMap.iloc[0]['count']],
        [dfAll.iloc[3]['count'], dfListe.iloc[3]['count'], dfChord.iloc[2]['count'], dfMap.iloc[1]['count']],
        [dfAll.iloc[4]['count'], dfListe.iloc[4]['count'], dfChord.iloc[3]['count'], dfMap.iloc[2]['count']],
        [dfAll.iloc[5]['count'], 0, dfChord.iloc[4]['count'], dfMap.iloc[3]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


,mood-delta,count
0,-1.0,7
1,0.0,69
2,1.0,15
3,2.0,1
